In [1]:
from jupyter_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd

from dash.dependencies import Input, Output

In [2]:
df = pd.read_csv("2020_LoL_esports_match_data_from_OraclesElixir_20210730.csv")

/home/tuedo/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.shape

(115140, 117)

In [4]:
print(df.columns.to_list())

['gameid', 'datacompleteness', 'url', 'league', 'year', 'split', 'playoffs', 'date', 'game', 'patch', 'playerid', 'side', 'position', 'player', 'team', 'champion', 'ban1', 'ban2', 'ban3', 'ban4', 'ban5', 'gamelength', 'result', 'kills', 'deaths', 'assists', 'teamkills', 'teamdeaths', 'doublekills', 'triplekills', 'quadrakills', 'pentakills', 'firstblood', 'firstbloodkill', 'firstbloodassist', 'firstbloodvictim', 'team kpm', 'ckpm', 'firstdragon', 'dragons', 'opp_dragons', 'elementaldrakes', 'opp_elementaldrakes', 'infernals', 'mountains', 'clouds', 'oceans', 'dragons (type unknown)', 'elders', 'opp_elders', 'firstherald', 'heralds', 'opp_heralds', 'firstbaron', 'barons', 'opp_barons', 'firsttower', 'towers', 'opp_towers', 'firstmidtower', 'firsttothreetowers', 'inhibitors', 'opp_inhibitors', 'damagetochampions', 'dpm', 'damageshare', 'damagetakenperminute', 'damagemitigatedperminute', 'wardsplaced', 'wpm', 'wardskilled', 'wcpm', 'controlwardsbought', 'visionscore', 'vspm', 'totalgold',

In [5]:
lcs_df = df[(df["league"] == "LCS")];
lcs_df.shape

(3168, 117)

In [6]:
lcs_pick_dict = dict()

for champ in lcs_df['champion']:
    lcs_pick_dict[champ] = lcs_pick_dict.get(champ,0) + 1
    
lcs_ban_dict = dict()

for ban in lcs_df[['ban1','ban2','ban3','ban4','ban5']]:
    for champ in lcs_df[ban]:
        lcs_ban_dict[champ] = lcs_ban_dict.get(champ,0) + 1

for key in lcs_ban_dict:
    lcs_ban_dict[key] = int(lcs_ban_dict[key] / 6)

lcs_pick_df = pd.DataFrame.from_dict({'champion' : list(lcs_pick_dict.keys()), 'number of picks' : list(lcs_pick_dict.values())})
lcs_pick_df = lcs_pick_df.dropna()
lcs_pick_df = lcs_pick_df.sort_values('number of picks', ascending = False)

lcs_ban_df = pd.DataFrame.from_dict({'champion' : list(lcs_ban_dict.keys()), 'number of bans' : list(lcs_ban_dict.values())})
lcs_ban_df = lcs_ban_df.dropna()
lcs_ban_df = lcs_ban_df.sort_values('number of bans', ascending = False)

lcs_pick_ban_df = pd.merge(lcs_pick_df, lcs_ban_df, how="outer", on=["champion"]).fillna(0)
lcs_pick_ban_df["pick and ban"] = lcs_pick_ban_df.loc[:,["number of picks", "number of bans"]].sum(axis=1)

# There were 264 games played in LCS 2020 (counted outside of this notebook)
lcs_pick_ban_df["P%"] = 100 * lcs_pick_ban_df["number of picks"] / 264
lcs_pick_ban_df["B%"] = 100 * lcs_pick_ban_df["number of bans"] / 264
lcs_pick_ban_df = lcs_pick_ban_df.sort_values("pick and ban", ascending=False)

In [7]:
lec_df = df[(df["league"] == "LEC")];
lec_df.shape

(2892, 117)

In [8]:
lec_pick_dict = dict()

for champ in lec_df['champion']:
    lec_pick_dict[champ] = lec_pick_dict.get(champ,0) + 1
    
lec_ban_dict = dict()

for ban in lec_df[['ban1','ban2','ban3','ban4','ban5']]:
    for champ in lec_df[ban]:
        lec_ban_dict[champ] = lec_ban_dict.get(champ,0) + 1

for key in lec_ban_dict:
    lec_ban_dict[key] = int(lec_ban_dict[key] / 6)

lec_pick_df = pd.DataFrame.from_dict({'champion' : list(lec_pick_dict.keys()), 'number of picks' : list(lec_pick_dict.values())})
lec_pick_df = lec_pick_df.dropna()
lec_pick_df = lec_pick_df.sort_values('number of picks', ascending = False)

lec_ban_df = pd.DataFrame.from_dict({'champion' : list(lec_ban_dict.keys()), 'number of bans' : list(lec_ban_dict.values())})
lec_ban_df = lec_ban_df.sort_values('number of bans', ascending = False)

lec_pick_ban_df = pd.merge(lec_pick_df, lec_ban_df, how="outer", on=["champion"]).fillna(0)
lec_pick_ban_df["pick and ban"] = lec_pick_ban_df.loc[:,["number of picks", "number of bans"]].sum(axis=1)

# There were 241 games played in LEC 2020 (counted outside of this notebook)
lec_pick_ban_df["P%"] = 100 * lec_pick_ban_df["number of picks"] / 241
lec_pick_ban_df["B%"] = 100 * lec_pick_ban_df["number of bans"] / 241

lec_pick_ban_df = lec_pick_ban_df.sort_values("pick and ban", ascending=False)

In [9]:
lec_pick_ban_df = lec_pick_ban_df.drop(labels=120, axis=0) # manually removing buggy entry

In [10]:
lck_df = df[(df["league"] == "LCK")];
lck_df.shape

(5676, 117)

In [11]:
lck_pick_dict = dict()

for champ in lck_df['champion']:
    lck_pick_dict[champ] = lck_pick_dict.get(champ,0) + 1
    
lck_ban_dict = dict()

for ban in lck_df[['ban1','ban2','ban3','ban4','ban5']]:
    for champ in lck_df[ban]:
        lck_ban_dict[champ] = lck_ban_dict.get(champ,0) + 1

for key in lck_ban_dict:
    lck_ban_dict[key] = int(lck_ban_dict[key] / 6)

lck_pick_df = pd.DataFrame.from_dict({'champion' : list(lck_pick_dict.keys()), 'number of picks' : list(lck_pick_dict.values())})
lck_pick_df = lck_pick_df.dropna()
lck_pick_df = lck_pick_df.sort_values('number of picks', ascending = False)

lck_ban_df = pd.DataFrame.from_dict({'champion' : list(lck_ban_dict.keys()), 'number of bans' : list(lck_ban_dict.values())})
lck_ban_df = lck_ban_df.dropna()
lck_ban_df = lck_ban_df.sort_values('number of bans', ascending = False)

lck_pick_ban_df = pd.merge(lck_pick_df, lck_ban_df, how="outer", on=["champion"]).fillna(0)
lck_pick_ban_df["pick and ban"] = lck_pick_ban_df.loc[:,["number of picks", "number of bans"]].sum(axis=1)

# There were 473 games played in LCK 2020 (counted outside of this notebook)
lck_pick_ban_df["P%"] = 100 * lck_pick_ban_df["number of picks"] / 473
lck_pick_ban_df["B%"] = 100 * lck_pick_ban_df["number of bans"] / 473
lck_pick_ban_df = lck_pick_ban_df.sort_values("pick and ban", ascending=False)

In [12]:
lpl_df = df[(df["league"] == "LPL")];
lpl_df.shape

(8712, 117)

In [13]:
lpl_pick_dict = dict()

for champ in lpl_df['champion']:
    lpl_pick_dict[champ] = lpl_pick_dict.get(champ,0) + 1
    
lpl_ban_dict = dict()

for ban in lpl_df[['ban1','ban2','ban3','ban4','ban5']]:
    for champ in lpl_df[ban]:
        lpl_ban_dict[champ] = lpl_ban_dict.get(champ,0) + 1

for key in lpl_ban_dict:
    lpl_ban_dict[key] = int(lpl_ban_dict[key] / 6)

lpl_pick_df = pd.DataFrame.from_dict({'champion' : list(lpl_pick_dict.keys()), 'number of picks' : list(lpl_pick_dict.values())})
lpl_pick_df = lpl_pick_df.dropna()
lpl_pick_df = lpl_pick_df.sort_values('number of picks', ascending = False)

lpl_ban_df = pd.DataFrame.from_dict({'champion' : list(lpl_ban_dict.keys()), 'number of bans' : list(lpl_ban_dict.values())})
lpl_ban_df = lpl_ban_df.dropna()
lpl_ban_df = lpl_ban_df.sort_values('number of bans', ascending = False)

lpl_pick_ban_df = pd.merge(lpl_pick_df, lpl_ban_df, how="outer", on=["champion"]).fillna(0)
lpl_pick_ban_df["pick and ban"] = lpl_pick_ban_df.loc[:,["number of picks", "number of bans"]].sum(axis=1)

# There were 726 games played in LPL 2020 (counted outside of this notebook)
lpl_pick_ban_df["P%"] = 100 * lpl_pick_ban_df["number of picks"] / 726
lpl_pick_ban_df["B%"] = 100 * lpl_pick_ban_df["number of bans"] / 726
lpl_pick_ban_df = lpl_pick_ban_df.sort_values("pick and ban", ascending=False)

In [14]:
# Setting up figures

lcs_picks = px.bar(lcs_pick_df, x="champion", y="number of picks");
lcs_bans = px.bar(lcs_ban_df, x="champion", y="number of bans");

lcs_pick_ban_percentage = px.bar(lcs_pick_ban_df, x="champion", y=["P%","B%"]);
lcs_pick_ban_percentage.update_layout(yaxis_title="P/B %", xaxis=dict(tickmode='linear'));
lcs_pick_ban_percentage.update_yaxes(range=[0,100]);


lec_picks = px.bar(lec_pick_df, x="champion", y="number of picks");
lec_bans = px.bar(lec_ban_df, x="champion", y="number of bans");

lec_pick_ban_percentage = px.bar(lec_pick_ban_df, x="champion", y=["P%","B%"]);
lec_pick_ban_percentage.update_layout(yaxis_title="P/B %", xaxis=dict(tickmode='linear'));
lec_pick_ban_percentage.update_yaxes(range=[0,100]);

lck_picks = px.bar(lck_pick_df, x="champion", y="number of picks");
lck_bans = px.bar(lck_ban_df, x="champion", y="number of bans");

lck_pick_ban_percentage = px.bar(lck_pick_ban_df, x="champion", y=["P%","B%"]);
lck_pick_ban_percentage.update_layout(yaxis_title="P/B %", xaxis=dict(tickmode='linear'));
lck_pick_ban_percentage.update_yaxes(range=[0,100]);

lpl_picks = px.bar(lpl_pick_df, x="champion", y="number of picks");
lpl_bans = px.bar(lpl_ban_df, x="champion", y="number of bans");

lpl_pick_ban_percentage = px.bar(lpl_pick_ban_df, x="champion", y=["P%","B%"]);
lpl_pick_ban_percentage.update_layout(yaxis_title="P/B %", xaxis=dict(tickmode='linear'));
lpl_pick_ban_percentage.update_yaxes(range=[0,100]);

In [18]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.H1('Lol Esports 2020 Visualizer'),
    
    html.H3('Pick-Ban by Region'),
    
    dcc.Dropdown(id='region-dropdown', options=[
        {'label' : 'LCS', 'value' : 'lcs'},
        {'label' : 'LEC', 'value' : 'lec'},
        {'label' : 'LCK', 'value' : 'lck'},
        {'label' : 'LPL', 'value' : 'lpl'},
    ], value='lcs',),
    
    html.Div(id='region-num-games'),
       
    dcc.Graph(
        id='pick-graph',
        figure=lcs_picks
    ),
    
        
    dcc.Graph(
        id='ban-graph',
        figure=lcs_bans
    ),
    
        
    dcc.Graph(
        id='p-b-percentage',
        figure=lcs_pick_ban_percentage
    ),
    
    html.Div("Match data taken from Oracle's Elixir")
])

@app.callback(
    Output('region-num-games', 'children'),
    [Input('region-dropdown', 'value')]
)
def update_num_games(value):
    if (value == 'lcs'):
        return 'There were 264 games played in LCS 2020.'
    elif (value == 'lec'):
        return 'There were 241 games played in LEC 2020.'
    elif (value == 'lck'):
        return 'There were 473 games played in LCK 2020.'
    elif (value == 'lpl'):
        return 'There were 726 games played in LPL 2020.'
    
@app.callback(
    Output('pick-graph', 'figure'),
    [Input(component_id='region-dropdown', component_property='value')]
)
def update_picks(value):
    if (value == 'lcs'):
        return lcs_picks
    elif (value == 'lec'):
        return lec_picks
    elif (value == 'lck'):
        return lck_picks
    elif (value == 'lpl'):
        return lpl_picks
@app.callback(
    Output('ban-graph', 'figure'),
    [Input(component_id='region-dropdown', component_property='value')]
)
def update_bans(value):
    if (value == 'lcs'):
        return lcs_bans
    elif (value == 'lec'):
        return lec_bans
    elif (value == 'lck'):
        return lck_bans
    elif (value == 'lpl'):
        return lpl_bans
@app.callback(
    Output('p-b-percentage', 'figure'),
    [Input(component_id='region-dropdown', component_property='value')]
)
def update_p_b_per(value):
    if (value == 'lcs'):
        return lcs_pick_ban_percentage
    elif (value == 'lec'):
        return lec_pick_ban_percentage
    elif (value == 'lck'):
        return lck_pick_ban_percentage
    elif (value == 'lpl'):
        return lpl_pick_ban_percentage

app.run_server(host="localhost",port=8051) # use whatever port is not 'in use', I'm not sure how this part works.

Dash app running on http://localhost:8051/
